In [86]:
import pandas as pd
import numpy as np
import os,sys

os.chdir('/data/arpit.goel/27_MarketBasketAnalysis/')

orders=pd.read_csv('02.ExtractedData/orders.csv')
products_prior=pd.read_csv('02.ExtractedData/order_products__prior.csv')
products_train=pd.read_csv('02.ExtractedData/order_products__train.csv')


## Predict order items based on past orders

In [10]:
user_products=pd.merge(pd.concat([products_prior,products_train]),orders[['order_id','user_id']],on='order_id')
user_products=user_products.groupby(['user_id','product_id'])['order_id'].size().reset_index()
user_products.columns=['user_id','product_id','count']
user_products.sort_values(by=['user_id','product_id','count'],ascending=[1,1,0],inplace=True)
user_products=user_products.groupby('user_id')['product_id'].apply(lambda x: ' '.join(map(str,x)))

In [23]:
count_products=pd.merge(pd.concat([products_prior,products_train]),orders[['order_id','user_id']],on='order_id')
count_products=count_products.groupby(['user_id','order_id'])['product_id'].size().reset_index()
count_products.columns=['user_id','order_id','product_id']
count_products['order_counts']=1
count_products=count_products.groupby(['user_id'])[['order_counts','product_id']].sum()
count_products['avg_products']=np.ceil(1.0*count_products['product_id']/count_products['order_counts'])

In [38]:
prediction_orders=orders[orders['eval_set']=='test']
prediction_orders['products']=prediction_orders['user_id'].map(user_products).fillna('None')
prediction_orders['num_products']=prediction_orders['user_id'].map(count_products['avg_products']).fillna(1)
prediction_orders['products']=prediction_orders[['products','num_products']].apply(lambda x: ' '.join(x[0].split(' ')[:int(x[1])]),axis=1)
prediction_orders[['order_id','products']].to_csv('03.BaselineResults/FavouriteUserProducts.csv',index=False)

/data/arpit.goel/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/data/arpit.goel/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/data/arpit.goel/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

## Predict order items based on past 10 orders

In [76]:
user_products=pd.merge(pd.concat([products_prior,products_train]),orders[['order_id','user_id']],on='order_id')

output=[]
for name, group in user_products.groupby('user_id'):
    last_10_orders=group.groupby('order_id')['order_id'].max().tail(10)
    #last_10_orders_products=pd.merge(group,last_10_orders,on='order_id')
    last_10_orders_products=group.join(last_10_orders,on='order_id',rsuffix='a')
    output_dict={}
    output_dict['user_id']=name
    avg_num_products=int(1.0*len(last_10_orders_products)/len(last_10_orders))
    output_dict['products_sorted']=' '.join(map(str,last_10_orders_products['product_id'].value_counts().index.tolist()[:avg_num_products]))
    output.append(output_dict)
    
output=pd.Series(output)

prediction_orders=orders[orders['eval_set']=='test']
prediction_orders['products']=prediction_orders['user_id'].map(output)
prediction_orders[['order_id','products']].to_csv('03.BaselineResults/FavouriteUserProductsP10Orders.csv',index=False)

/data/arpit.goel/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [88]:
orders=orders.sort_values(by=['user_id','order_number'])
orders['day_count']=orders.fillna(0).groupby('user_id')['days_since_prior_order'].apply(np.cumsum)

In [95]:
users_freq=pd.crosstab(orders['user_id'],orders['eval_set'])

In [100]:
users_freq[users_freq['prior']==3]

eval_set,prior,test,train
user_id,,,
6,3,1,0
8,3,0,1
9,3,0,1
25,3,1,0
33,3,1,0
44,3,0,1
51,3,1,0
53,3,0,1
106,3,0,1


In [116]:
aisles=pd.read_csv('02.ExtractedData/aisles.csv')
products=pd.read_csv('02.ExtractedData/products.csv')
departments=pd.read_csv('02.ExtractedData/departments.csv')
eg=pd.merge(user_products[user_products['user_id']==51],products,on='product_id')
eg=pd.merge(eg,aisles,on='aisle_id')
eg=pd.merge(eg,departments,on='department_id')
eg.sort_values(by=['order_id','add_to_cart_order'])

,order_id,product_id,add_to_cart_order,reordered,user_id,product_name,aisle_id,department_id,aisle,department
0,1099991,10017,1,0,51,Tilapia Filet,39,12,seafood counter,meat seafood
2,1099991,40604,2,0,51,Feta Cheese Crumbles,21,16,packaged cheese,dairy eggs
13,1099991,30489,3,0,51,Original Hummus,67,20,fresh dips tapenades,deli
16,1099991,44359,4,0,51,Organic Small Bunch Celery,83,4,fresh vegetables,produce
22,1099991,24507,5,0,51,Sun-Ripened Dried Tomatoes,81,15,canned jarred vegetables,canned goods
14,1099991,45302,6,0,51,Mini Falafel With Tahini Sauce,1,20,prepared soups salads,deli
24,1099991,445,7,0,51,Almond Honey Protein+ Bar,3,19,energy granola bars,snacks
25,1099991,20115,8,0,51,Organic Trail Mixer Gluten Free Chewy Granola ...,3,19,energy granola bars,snacks
26,1099991,5463,9,0,51,Original Oat Bar,3,19,energy granola bars,snacks
5,1099991,33000,10,0,51,Pure Irish Butter,36,16,butter,dairy eggs


In [117]:
orders[orders['user_id']==51]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,day_count
747,1099991,51,prior,1,3,14,NaN,0
748,1505429,51,prior,2,2,14,13,13
749,2151817,51,prior,3,5,14,17,30
750,2174416,51,test,4,0,14,16,46
